In [4]:
import torch
from torch.masked import masked_tensor, as_masked_tensor
import warnings

# Disable prototype warnings
warnings.filterwarnings(action='ignore', category=UserWarning)


In [5]:
data = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=torch.float32)
mask = torch.tensor([[1, 1, 0], [1, 0, 1], [0, 1, 1]], dtype=torch.bool)
mt = masked_tensor(data, mask)


In [6]:
data_tensor = mt.get_data()
mask_tensor = mt.get_mask()

data = torch.arange(24).reshape(2, 3, 4)
mask = data % 2 == 0
mt = masked_tensor(data.float(), mask)

print("mt[0]:\n", mt[0])
print("mt[:, :, 2:4]:\n", mt[:, :, 2:4])


mt[0]:
 MaskedTensor(
  [
    [  0.0000,       --,   2.0000,       --],
    [  4.0000,       --,   6.0000,       --],
    [  8.0000,       --,  10.0000,       --]
  ]
)
mt[:, :, 2:4]:
 MaskedTensor(
  [
    [
      [  2.0000,       --],
      [  6.0000,       --],
      [ 10.0000,       --]
    ],
    [
      [ 14.0000,       --],
      [ 18.0000,       --],
      [ 22.0000,       --]
    ]
  ]
)


In [7]:
torch.manual_seed(0)
# This would be the type of data that you would get as an embedding
kernel = torch.ones(1, 1, 2, 2)
data = torch.arange(16).reshape(1, 1, 4, 4).float()
mask = torch.randint(0, 2, (1, 1, 4, 4), dtype=torch.bool)
mt = masked_tensor(data, mask)

# Extracting, unfolding and reshaping
extracted_data = mt.get_data()
extracted_mask = mt.get_mask()
kernel = kernel.contiguous().reshape(1, 1, 4)
# Unfolding
unfolded_data = extracted_data.unfold(2, 2, 1).unfold(3, 2, 1).contiguous().reshape(1, 1, 9, 4)
unfolded_mask = extracted_mask.unfold(2, 2, 1).unfold(3, 2, 1).contiguous().reshape(1, 1, 9, 4)

# New masked tensor, matmul is not supported yet so no way to really do it
mt = masked_tensor(unfolded_data, unfolded_mask)
result = mt.mul(kernel)
print(result)

MaskedTensor(
  [
    [
      [
        [      --,   1.0000,   4.0000,   5.0000],
        [  1.0000,   2.0000,   5.0000,   6.0000],
        [  2.0000,       --,   6.0000,   7.0000],
        [  4.0000,   5.0000,   8.0000,   9.0000],
        [  5.0000,   6.0000,   9.0000,  10.0000],
        [  6.0000,   7.0000,  10.0000,       --],
        [  8.0000,   9.0000,       --,  13.0000],
        [  9.0000,  10.0000,  13.0000,       --],
        [ 10.0000,       --,       --,       --]
      ]
    ]
  ]
)


In [17]:
import time
import torch
# Creating a mask tensor fully masked
torch.manual_seed(0)
# This would be the type of data that you would get as an embedding
kernel = torch.ones(1, 1, 2, 2)
data = torch.arange(65536).reshape(1, 1, 256, 256).float()
mask = torch.arange(65536).reshape(1, 1, 256, 256).bool()
mt = masked_tensor(data, ~mask)
print(mt)
# Calculating the time it takes to element-wise multiplication
time_start = time.time()
mt = mt.mean()
end_time = time.time()
print("Time taken for mean Masked: ", end_time - time_start)
time_start = time.time()
data = data.mean()
end_time = time.time()
print("Time taken for mean: ", end_time - time_start)


MaskedTensor(
  [
    [
      [
        [  0.0000,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,       --,